In [12]:
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob
from sklearn.model_selection import train_test_split
from statistics import mean
global train_x , train_y,x_test,y_test,TimeData,Type
train_x=[]
train_y=[]
x_test=[]
y_test=[]

def Resha(x):
    nptrain=np.array(x)
    nptrain=np.reshape(nptrain,(nptrain.shape[0]*nptrain.shape[1], nptrain.shape[2]))
    return nptrain

def Savenp(x,y):
    train_x,x_test,train_y,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    StockName=path[12:21]
    
    Npdata=Resha(train_x)
    np.save(os.path.join('./StockData/TrainingData/','NormtrainingX_'+StockName),Npdata)
    print(path[12:21]," trainX  ",Npdata.shape)
    
    Npdata=Resha(x_test)
    np.save(os.path.join('./StockData/TrainingData/','NormtestingX_'+StockName),Npdata)
    print(path[12:21]," testX  ",Npdata.shape)
    
    Npdata=np.array(train_y)
    np.save(os.path.join('./StockData/TrainingData/','trainingY_'+StockName),Npdata)
    print(path[12:21]," trainY  ",Npdata.shape)
    
    Npdata=np.array(y_test)
    np.save(os.path.join('./StockData/TrainingData/','testingY_'+StockName),Npdata)
    print(path[12:21]," testY  ",Npdata.shape)

def add_features(data):
    #kd   rsv=(今天收盤-最近9天的最低價)/(最近9天的最高價-最近9天的最低價)
    rsv=[] 
    k=[] 
    d=[]
    close=data.loc[ : ,'close'].values.tolist()#print(close)
    for i in range(0,len(close)):
        if i>=8:
            pre9low=data.loc[ i-8 : i ,'low'].values.tolist()
            low=min(pre9low)
            pre9high=data.loc[ i-8 : i ,'high'].values.tolist()
            high=max(pre9high)#print((close[i]-low)/(high-low))
            rsv.append(((close[i]-low)/(high-low))*100)
        else:
            rsv.append(0)   
            
    for i in range(0,len(rsv)):
        if i>=1:
            k.append(((2/3)*k[i-1])+((1/3)*rsv[i]))
            d.append(((2/3)*d[i-1])+((1/3)*k[i]))
        else:
            k.append(rsv[0])
            d.append(k[0])
            
    data['k']=k
    data['d']=d
    data['rsv']=rsv
    return data

def GenerateTrain(Type,feature):
    for tmp in TimeData:#print(tmp[1])
        i=tmp[1].dropna()
        i=i.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))#print(i)
        if len(i)==5: #Decide the way seperate the stock data
            train_x.append(i.loc[ : ,feature].values.tolist())# print(train_x)
            Open=i.loc[ : ,'open'].values.tolist()[0]
            Close=i.loc[ : ,Type].values.tolist()[len(i.loc[ : ,Type].values.tolist())-1]
            train_y.append(float(Close-Open))
        else:
            continue
    Savenp(train_x,train_y)

feature=input("Enter the needed features seperated with space: ").split(" ")
closeType=input("Enter the close type: ")
resam=input("Enter the desired  data span , d  for day  w  for week  m  for month: ")
#span=int(input("Enter the span of trainx trainy: "))|

for path in glob.glob(r'./StockData/stock0050.csv'):
    CsvData=pd.DataFrame(pd.read_csv(path))
    CsvData['date']=pd.to_datetime(CsvData['date'])
    CsvData=CsvData.drop([0],axis=0)#drop 第一天 因為stockdata 有16年跳到17年的問題
    CsvData = add_features(CsvData)#print(CsvData)
    TimeData=CsvData.set_index('date').resample(resam)#group
    GenerateTrain(closeType,feature)
    train_x.clear()
    train_y.clear()

Enter the needed features seperated with space: close open low high volume k d rsv
Enter the close type: adj_close
Enter the desired  data span , d  for day  w  for week  m  for month: w
stock0050  trainX   (2515, 8)
stock0050  testX   (840, 8)
stock0050  trainY   (503,)
stock0050  testY   (168,)
